## pip

In [1]:
!pip install datasets

In [ ]:
!pip install tiktoken

## Import

In [2]:
import os
import re
import time
import json
import random
import string
import psutil
import pickle
from tqdm import tqdm
from pprint import pprint
from collections import Counter, defaultdict

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import mode

from datasets import load_dataset
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, normalizers, decoders, processors
import tiktoken

import torch
from torch.utils.data import TensorDataset, Dataset, IterableDataset, DataLoader

In [3]:
from IPython.display import HTML
shell = get_ipython()

def adjust_font_size():
  display(HTML('''<style>
    body {
      font-size: 24px;
    }
  '''))

if adjust_font_size not in shell.events.callbacks['pre_execute']:
  shell.events.register('pre_execute', adjust_font_size)

# 🔴 **Utils**

In [4]:
def prepare_data(tokens, seq_len):
    # Trim tokens so that total length is divisible by seq_len
    n_tokens = (tokens.shape[0] // seq_len) * seq_len
    tokens = tokens[:n_tokens]

    # Reshape to 2D tensor
    return tokens.view(-1, seq_len)


In [5]:
def num_trainable_params(model):
  nums = sum(p.numel() for p in model.parameters() if p.requires_grad)/1e6
  return nums

In [6]:
def calculate_time(model, x, num_runs=10):
    torch.cuda.synchronize()
    start = time.time()
    for _ in range(num_runs):
        model(*x)
    torch.cuda.synchronize()
    return (time.time() - start) / num_runs


def calculate_time_cpu(model, x, num_runs=10):
    start = time.time()
    for _ in range(num_runs):
        model(*x)
    return (time.time() - start) / num_runs

# 🟥 tokenize Tiktoken fast

In [ ]:
dataset=load_dataset("roneneldan/TinyStories")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
dataset.shape

{'train': (2119719, 1), 'validation': (21990, 1)}

In [ ]:
tokenizer=tiktoken.get_encoding("gpt2")
tokenized_train_samples = []
for item in tqdm(dataset["train"], desc="Tokenizing Train Set"):
    input_ids = tokenizer.encode(item["text"])
    tokenized_train_samples.append(np.array(input_ids))

In [ ]:
tokenized_valid_samples = []
for item in tqdm(dataset["validation"], desc="Tokenizing validation Set"):
    input_ids = tokenizer.encode(item["text"])
    tokenized_valid_samples.append(np.array(input_ids))

In [ ]:
tokenized_valid_samples[:1]

In [ ]:
sumtoks=  sum(len(tok) for tok in tokenized_train_samples)
print(sumtoks)

# 🟥 Train Bpe Tokenizer and data loader

## 🟧 BPE Trainer

In [ ]:
# Initialize a BPE tokenizer
tokenizer = Tokenizer(models.BPE(unk_token="|<unk>|"))

# Use a pre-tokenizer to split text into words
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

# Initialize a BPE trainer
trainer = trainers.BpeTrainer(
    vocab_size=10_000,  # Set the vocabulary size
    special_tokens=["|<unk>|", "<|endoftext|>"],
    min_frequency=2,  # Set the minimum frequency of tokens
    )

# Train the tokenizer on a custom dataset
tokenizer.train_from_iterator(dataset["train"]["text"], trainer)

# Add special tokens
tokenizer.post_processor = processors.TemplateProcessing(
    single="<|endoftext|> $A",
    special_tokens=[("<|endoftext|>", tokenizer.token_to_id("<|endoftext|>"))],
)

# Add decoder
tokenizer.decoder = decoders.ByteLevel(add_prefix_space=False)

# Save the trained tokenizer
tokenizer.save("bpe-tokenizer_tinystories.json")

#
print(f"🎉 Tokenizer training complete!")
print(f"🔹 Vocabulary size: {tokenizer.get_vocab_size():,} tokens")

🎉 Tokenizer training complete!
🔹 Vocabulary size: 10,000 tokens


In [ ]:
# Initialize a BPE tokenizer
tokenizer = Tokenizer.from_file("bpe-tokenizer_tinystories.json")
print(f"🎉 Tokenizer training complete!")
print(f"🔹 Vocabulary size: {tokenizer.get_vocab_size():,} tokens")

🎉 Tokenizer training complete!
🔹 Vocabulary size: 10,000 tokens


In [88]:
sent = 'They played together all day and became best friends.'
tokens = tokenizer.encode(sent)
print(tokens.ids)
print(tokens.tokens)

pprint(tokenizer.decode(tokens.ids))

[1, 546, 667, 462, 378, 252, 161, 1042, 725, 375, 15]
['<|endoftext|>', 'They', 'Ġplayed', 'Ġtogether', 'Ġall', 'Ġday', 'Ġand', 'Ġbecame', 'Ġbest', 'Ġfriends', '.']
'They played together all day and became best friends.'


## 🟧 Save and load Tokens with BPE tokenizer

In [ ]:
# Tokenization {train}
tokenized_train_samples = []
for item in tqdm(dataset["train"], desc="Tokenizing Train Set"):
    input_ids = tokenizer.encode(item["text"]).ids
    tokenized_train_samples.append(np.array(input_ids))

In [ ]:
tokenized_train_samples_concat=[]
tokenized_train_samples_concat = np.concatenate(tokenized_train_samples)
len(tokenized_train_samples_concat)

In [ ]:
# Save tokens as a pytorch file
torch.save(torch.tensor(tokenized_train_samples_concat), 'tokenized-train-samples_vocab-10k.pt')

In [ ]:
# Tokenization {validation}
tokenized_valid_samples = []
for item in tqdm(dataset["validation"], desc="Tokenizing Validation Set"):
    input_ids = tokenizer.encode(item["text"]).ids
    tokenized_valid_samples.append(np.array(input_ids))

In [ ]:
tokenized_valid_samples_concat=[]
tokenized_valid_samples_concat = np.concatenate(tokenized_valid_samples)
len(tokenized_valid_samples_concat)

In [ ]:
# Save tokens as a pytorch file
torch.save(torch.tensor(tokenized_valid_samples_concat), 'tokenized-valid-samples_vocab-10k.pt')

## 🟧 Custom dataset

In [9]:
class TinyStoriesDataset(Dataset):

    def __init__(self, data, seq_len):
        self.seq_len = seq_len
        self.data = prepare_data(data, seq_len+1)

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        sample = self.data[idx]
        return sample[:-1], sample[1:]

In [10]:
train_set = TinyStoriesDataset(tokenized_train_samples, 128)
train_set.data.shape, len(train_set), train_set[0]

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
%timeit next(iter(train_set))

In [ ]:
b=next(iter(train_set))

In [ ]:
len(b)

## 🟧 DataLoader

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tokenizer = Tokenizer.from_file("/content/drive/MyDrive/temp/bpe-tokenizer_tinystories.json")
tokenizer

In [13]:
# tokenized_train_samples = torch.load('/content/drive/MyDrive/temp/tokenized-train-samples_vocab-10k.pt')
tokenized_valid_samples = torch.load('/content/drive/MyDrive/temp/tokenized-valid-samples_vocab-10k.pt')

# train_set = TinyStoriesDataset(tokenized_train_samples, seq_len=128)
valid_set = TinyStoriesDataset(tokenized_valid_samples, seq_len=128)

In [14]:
# train_loader = DataLoader(train_set, batch_size=32, shuffle=True, pin_memory=True) #, num_workers=2)
valid_loader = DataLoader(valid_set, batch_size=32, shuffle=False, pin_memory=True) #, num_workers=2)

In [15]:
x_batch, y_batch = next(iter(valid_loader))
x_batch.shape, y_batch.shape

(torch.Size([32, 128]), torch.Size([32, 128]))

In [ ]:
print(x_batch[0,:])
print('\n',y_batch[0,:])

In [ ]:
len(train_loader), len(valid_loader)

In [57]:
len(train_loader) / (20*60)

93.865

In [18]:
train_iter = iter(train_loader)

In [ ]:
%timeit next(train_iter)

## 🟧 EDA

In [16]:
token_count_stories=[]
for tokns in tokenized_train_samples:
    token_count_stories.append(len(tokns))

In [ ]:
token_count_stories_np=np.array(token_count_stories)

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(token_count_stories, bins=50, kde=True)
plt.xlabel('Token Count')
plt.ylabel('Frequency')
plt.title('Distribution of Token Counts')
plt.show()

In [ ]:
np.sort(token_count_stories_np)[:1000]

# 🟥 Transformer Model from scratch

In [17]:
class MultiHeadAtention(torch.nn.Module):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.qkv_proj = torch.nn.Linear(embed_dim, 3 * embed_dim)
        self.out_proj = torch.nn.Linear(embed_dim, embed_dim)
    # احتمالا خطا ابعاد دارد زمان ترین ممکن است ترین نشود
    def forward(self, x):
        batch_size, seq_len, embed_dim = x.size()
        k,q,v = self.qkv_proj(x).view(batch_size, seq_len, 3, self.num_heads, self.head_dim).transpose(1,2).chunk(3)
        # F.scaled_dot_product_attention(q,k,v)
        # return self.out_proj(x)
        return q

In [ ]:
x=torch.range(1,24).view(2,3,4)
print(x)

# /x=x.transpose(1,0)
print(x)
#

In [ ]:
print(x.shape)
y= MultiHeadAtention(4,2)(x)
y.shape

# 🔴 **Model from scratch - Howsam**

## 🟠 Define

In [21]:
import time
from dataclasses import dataclass

from datasets import load_dataset
from tokenizers import Tokenizer

import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.nn import functional as F

In [22]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

## 🟠 Embedding

In [23]:
wte = nn.Embedding(tokenizer.get_vocab_size(), 100)
wte(torch.tensor([1, 2,3])).shape

torch.Size([3, 100])

In [24]:
seq_len = 128
wpe = nn.Embedding(seq_len, 100)
wpe(torch.tensor([1, 2, 100])).shape

torch.Size([3, 100])

In [113]:
wpe(torch.arange(x_batch.shape[1])).shape

torch.Size([128, 100])

In [25]:
x = wte(x_batch) + wpe(torch.arange(x_batch.shape[1]))
x.shape

torch.Size([32, 128, 100])

## 🟠 Scaled Dot-Product Attention

In [16]:
q = k = v = x
print(q.shape)

mask = torch.tril(torch.ones(seq_len, seq_len))

scores = q @ k.transpose(-2, -1) / (k.shape[-1]**0.5)
scores.masked_fill_(mask ==0, float(-torch.inf))
scores = scores.softmax(dim=-1)
print(scores.shape)

z = scores @ v
z.shape

NameError: name 'x' is not defined

In [ ]:
# scores = torch.randn(3, 5, 5)
# mask = torch.tril(torch.ones(5, 5))
# scores.masked_fill_(mask ==0, float(-torch.inf))
# scores = scores.softmax(dim=-1)
# scores

In [ ]:
def scaled_dot_product_attention(q, k, v):
    mask = torch.tril(torch.ones(q.shape[-2], q.shape[-2])).to(device)
    scores = q @ k.transpose(-2, -1) / (k.shape[-1]**0.5)
    scores.masked_fill_(mask==0, float(-torch.inf))
    scores = scores.softmax(dim=-1)
    z = scores @ v
    return z

In [ ]:
scaled_dot_product_attention(x.to(device), x.to(device), x.to(device)).shape

In [17]:
q = torch.randn((128, 1024, 768), device=device)
k = torch.randn((128, 1024, 768), device=device)
v = torch.randn((128, 1024, 768), device=device)
q.shape

torch.Size([128, 1024, 768])

In [18]:
scaled_dot_product_attention(q, k, v).shape

NameError: name 'scaled_dot_product_attention' is not defined

In [ ]:
# calculate_time(scaled_dot_product_attention, (q, k, v), num_runs=20)
calculate_time_cpu(scaled_dot_product_attention, (q, k, v), num_runs=20)

In [ ]:
F.scaled_dot_product_attention(q, k, v, is_causal=True).shape

In [ ]:
torch.abs(scaled_dot_product_attention(q, k, v) - F.scaled_dot_product_attention(q, k, v, is_causal=True)).max()

In [ ]:
calculate_time(F.scaled_dot_product_attention, (q, k, v), num_runs=20)

## 🟠 Multi Head Attention

In [ ]:
# class MultiHeadAttention(nn.Module):

#     def __init__(self):
#         super().__init__()
#         self.fc1 = nn.Linear(100, 1000)
#         self.fc2 = nn.Linear(1000, 100)
#         self.fc3 = nn.Linear(1000, 100)

#     def forward(self, x):
#         y = F.relu(self.fc1(x))
#         y1 = self.fc2(y)
#         y2 = self.fc3(y)
#         return F.relu(torch.concat([y1, y2], dim=-1))

In [ ]:
# mha = MultiHeadAttention()
# num_trainable_params(mha)
# mha.forward(torch.rand(10, 100)).shape

In [83]:
# x=torch.randn(2,4)
# print(x)
# lx=nn.Linear(4,8,bias=False)
# y1 =x@ lx.weight.T
# y2=lx(x)
# print(y1.softmax(dim=-1).argmax(dim=0))
# print(y1.softmax(dim=-1))
# print(lx.weight.T.shape)

In [19]:
class GPTConfig:
    n_embd: int = 100
    n_head: int = 5

config = GPTConfig()
config.n_embd

100

In [26]:
class MultiHeadAttention(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.n_embd = config.n_embd
        self.n_head = config.n_head
        self.head_size = self.n_embd // self.n_head

        self.qkv_proj = nn.Linear(self.n_embd, 3*self.n_embd, bias=False)

        self.c_proj = nn.Linear(self.n_embd, self.n_embd, bias=False)
        self.c_proj.residual = True

    def forward(self, x):
        B, T, C = x.shape
        q, k, v = self.qkv_proj(x).view(B, T, 3*self.n_head, self.head_size).transpose(1, 2).chunk(3, dim=-3)

        y = F.scaled_dot_product_attention(q, k, v, is_causal=True)

        y = y.transpose(1, 2).contiguous().view(B, T, C)

        y = self.c_proj(y)
        return y #,q, k, v

In [ ]:
# mha = MultiHeadAttention(config)
# y,q, k, v= mha(x)
# print("X:",x.shape)
# print("qkv_proj:",mha.qkv_proj.weight.T.shape)
# print("c_proj:",mha.c_proj.weight.T.shape)

# print("q:",q.shape)
# print(k.shape)
# print(v.shape)
# print("y:",y.shape)

In [ ]:
calculate_time(mha.to(device), (x.to(device),), num_runs=20)

## 🟠 Feed Forward (MLP)

In [21]:
class GPTConfig:
    n_embd: int = 100
    n_head: int = 5
    f_expnd: float = 4

config = GPTConfig()
config.n_embd

100

In [22]:
class FeedForward(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.n_embd = config.n_embd
        self.f_expnd = config.f_expnd

        self.up_proj = nn.Linear(self.n_embd, int(self.f_expnd*self.n_embd), bias=False)
        self.down_proj = nn.Linear(int(self.f_expnd*self.n_embd), self.n_embd, bias=False)
        self.down_proj.residual = True

    def forward(self, x):
        return self.down_proj(F.gelu(self.up_proj(x)))

In [ ]:
# x1=torch.randn(2,4,100)
# plt.hist(x1.flatten(), bins=50);
# plt.show()
# plt.hist(F.gelu(x1).flatten(), bins=50);
# plt.show()
# F.gelu(x1).min(dim=-1)

In [23]:
feedfor = FeedForward(config)
print(feedfor(x).shape)
feedfor.up_proj.weight.T.shape

NameError: name 'x' is not defined

In [25]:
num_trainable_params(feedfor)*1000

80.0

In [ ]:
calculate_time(mlp, (x, ), num_runs=20)

## 🟠 Decoder Block

In [28]:
class DecoderBlock(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.n_embd = config.n_embd

        self.ln1 = nn.LayerNorm(config.n_embd)
        self.mha = MultiHeadAttention(config)

        self.ln2 = nn.LayerNorm(config.n_embd)
        self.mlp = FeedForward(config)

    def forward(self, x):
        x = x + self.mha(self.ln1(x))
        x = x + self.mlp(self.ln2(x))
        return x

In [29]:
decoder = DecoderBlock(config)
y = decoder(x)
print(x.shape)
plt.hist(x.detach().flatten(), bins=50);
plt.show()
plt.hist(y.detach().flatten(), bins=50);
plt.show()

means_before = x.mean(dim=-1)  # shape: (32, 128)
means_after = y.mean(dim=-1)

stds_before = x.std(dim=-1)
stds_after = y.std(dim=-1)

plt.hist(means_before.detach().flatten(), bins=50);
plt.show()
plt.hist(means_after.detach().flatten(), bins=50);
plt.show()

NameError: name 'config' is not defined

In [ ]:
norml=nn.LayerNorm(100)
xrnd=torch.range(1,100)
print(xrnd.min().item(), xrnd.max().item(), xrnd.mean().item(), xrnd.std().item())
print(norml(xrnd).min().item(), norml(xrnd).max().item(),norml(xrnd).mean().item(),9 , norml(xrnd).std().item())
plt.hist(xrnd, bins=10);
plt.show()
plt.hist(norml(xrnd).detach(), bins=10);
plt.show()

In [30]:
num_trainable_params(decoder) * 1e3

NameError: name 'decoder' is not defined

In [49]:
calculate_time_cpu(decoder, (x, ), num_runs=20) * 1e3

33.37346315383911

## 🟠 GPT

In [31]:
class GPTConfig:
    vocab_size: int = 10_000
    seq_len: int = 128
    n_layer: int = 12
    n_embd: int = 100
    n_head: int = 5
    f_expnd: float = 4


config = GPTConfig()
config.n_embd

100

In [32]:
class GPT(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.config = config
        self.wte = nn.Embedding(config.vocab_size, config.n_embd)
        self.wpe = nn.Embedding(config.seq_len, config.n_embd)
        # self.decoders = nn.Sequential(*[DecoderBlock(config) for _ in range(config.n_layer)])
        self.decoders = nn.ModuleList([DecoderBlock(config) for _ in range(config.n_layer)])
        self.lnf = nn.LayerNorm(config.n_embd)
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        self.lm_head.weight = self.wte.weight
        # self.lm_head.weight.data.uniform_(-1/self.lm_head.in_features**0.5, 1/self.lm_head.in_features**0.5)
        # nn.init.uniform_(self.lm_head.weight, -1/self.lm_head.in_features**0.5, 1/self.lm_head.in_features**0.5)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        std = 0.02
        if isinstance(module, nn.Linear):
            if hasattr(module, 'residual'):
                std *= (2*self.config.n_layer)**-0.5
            nn.init.normal_(module.weight, mean=0.0, std=std)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            nn.init.normal_(module.weight, mean=0.0, std=std)

    def forward(self, idx):
        B, T = idx.shape

        x = self.wte(idx) + self.wpe(torch.arange(T, device=device))

        # x = self.decoders(x)
        for decoder in self.decoders:
            x = decoder(x)

        x = self.lnf(x)
        logits = self.lm_head(x)
        return logits

In [33]:
model = GPT(config).to(device)
logits= model(x_batch.to(device))
logits.shape

NameError: name 'FeedForward' is not defined

🦾 تمرین:

---


بخش تولید متن را جنریت و برای مدل جی پی تی به شکلی ساده و بر اساس آرگ مکس پیاده‌سازی کنید

In [34]:
gen_tokens= torch.argmax(F.softmax(logits,dim=-1), dim=-1)
pprint(tokenizer.decode(x_batch[0,:].tolist()))
print('\n\n')
pprint(tokenizer.decode(gen_tokens[0,:].tolist()))

NameError: name 'logits' is not defined

In [35]:
model.lm_head.weight.T.shape , model.wte.weight.T.shape

NameError: name 'model' is not defined

In [32]:
num_trainable_params(model), num_trainable_params(model.decoders), num_trainable_params(model.lm_head)

(2.4578, 1.4448, 1.0)

In [ ]:
calculate_time(model, (x_batch.to(device),), num_runs=100) * 1e3

## 🟠 Initialization

In [34]:
model = GPT(
    GPTConfig).to(device)

In [ ]:
plt.hist(model.decoders[0].mha.c_proj.weight.flatten().detach().cpu(), bins=50);

In [ ]:
0.02 * (2*4)**-0.5 * 3

0.021213203435596427

In [ ]:
plt.hist(model.wpe.weight.flatten()[:100_000].detach().cpu(), bins=50);

In [ ]:
plt.hist(model)

In [ ]:
plt.hist(model.decoders[2].mlp.down_proj.weight.flatten().detach().cpu(), bins=50);

# 🟥 GPT model implement with nn.torch transformer

🦾 تمرین


شبکه جی پی تی را با استفاده از دستورات آماده پای تورچ پیاده‌سازی کنید و زمان اجرای آن را با مدل پیاده‌سازی‌شده از صفر مقایسه کنید.

In [92]:
transformer = torch.nn.Transformer(d_model=100, nhead=5, num_encoder_layers=0, num_decoder_layers=12, dim_feedforward=400,
                     dropout=0.1, activation="gelu", custom_encoder=None, custom_decoder=None, layer_norm_eps=1e-05,
                     batch_first=False, norm_first=False, bias=True, device=None, dtype=None)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [97]:
decoder_layer = nn.TransformerDecoderLayer(d_model= 100, nhead=5, dim_feedforward=400)
transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=12)
memory = torch.rand(10, 32, 100)
tgt = torch.rand(20, 32, 100)
out = transformer_decoder(tgt, memory)
out.shape

torch.Size([20, 32, 100])

In [51]:
class GPTTorch(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.config = config
        self.wte = nn.Embedding(config.vocab_size, config.n_embd)
        self.wpe = nn.Embedding(config.seq_len, config.n_embd)
        decoder_layer = nn.TransformerDecoderLayer(d_model= config.n_embd, nhead=config.n_head,
                                                   dim_feedforward=config.f_expnd*config.n_embd,activation=F.gelu,norm_first=True
                                                   )
        # decoder_layer.self_attn.is_causal = True
        self.decoders = nn.TransformerDecoder(decoder_layer, num_layers=12)
        # self.decoders = nn.ModuleList([DecoderBlock(config) for _ in range(config.n_layer)])
        self.lnf = nn.LayerNorm(config.n_embd)
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        self.lm_head.weight = self.wte.weight

        self.apply(self._init_weights)

    def _init_weights(self, module):
        std = 0.02
        if isinstance(module, nn.Linear):
            if hasattr(module, 'residual'):
                std *= (2*self.config.n_layer)**-0.5
            nn.init.normal_(module.weight, mean=0.0, std=std)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            nn.init.normal_(module.weight, mean=0.0, std=std)

    def forward(self, idx):
        B, T = idx.shape

        x = self.wte(idx) + self.wpe(torch.arange(T, device=device))

        # x = self.decoders(x)
        # for decoder in self.decoders:
        x = self.decoders(x ,memory=None)

        x = self.lnf(x)
        logits = self.lm_head(x)
        return logits

In [52]:
model = GPTTorch(config).to(device)
logits= model(x_batch.to(device))
logits.shape

AttributeError: 'NoneType' object has no attribute 'is_nested'

In [109]:
gen_tokens= torch.argmax(F.softmax(logits,dim=-1), dim=-1)
pprint(tokenizer.decode(x_batch[0,:].tolist()))
print('\n\n')
pprint(tokenizer.decode(gen_tokens[0,:].tolist()))

('Spot. Spot saw the shiny car and said, "Wow, Kitty, your car is so bright '
 'and clean!" Kitty smiled and replied, "Thank you, Spot. I polish it every '
 'day."\n'
 '\n'
 'After playing with the car, Kitty and Spot felt thirsty. They found a small '
 'pond with clear water. They drank the water and felt very happy. They played '
 'together all day and became best friends.Once upon a time, in a big forest, '
 'there lived a rhinoceros named Roxy. Roxy loved to climb. She climbed trees, '
 'rocks, and hills. One day, Roxy found an icy hill. She')



(' throws singer becoming disgust mouse disgust disgust His earn farm ideas '
 'disgustothyextext farmendant farm farm farm farmendantendantendant opera smo '
 'farm glided intelligink donâiments intellig smo smoimentsœWhereœWhere '
 'plantroom att bru rang rang donâ donâ att successful promisedext meowing '
 'slower as blood h att blood hhen cuts h htedink hhenhenMaya exc beetle att '
 'slower gap h h Carl asounced hawkhenhenlephantext as

🦾 تمرین

کانفیگ مدل‌های تاینی استوری را در یک جدول خلاصه کنید و براساس این کانفیگ‌ها مدل را بسازید.

Howsam |vocab_size:10_000| seq_len:128| n_layer:12| n_embd:100|n_head:5| f_expnd: 4  

GPT-14 |vocab_size:50304| seq_len:1024| n_layer:12| n_embd:768|n_head:12| f_expnd: 4| dropout: 0.0| bias:True

gpt_neo | vocab_size:50257| seq_len:256| n_layer:8| n_embd:256|n_head:16| f_expnd: --| dropout: 0.1| torch_dtype:float16


🦾  تمرین


البته! در اینجا خلاصه‌ای از **ایده‌های اصلی** سه مدل معروف از دو خانواده مختلف آورده شده است:

---

## 🔹 خانواده Encoder-only (مدل‌هایی مثل BERT)

### 📌 1. **BERT (Bidirectional Encoder Representations from Transformers)**

* **نوع**: Encoder-only
* **ایده اصلی**: یادگیری نمایش‌های عمیق زبانی از متن با خواندن **دو طرفه** (bidirectional).
* **روش آموزش**:

  * **Masked Language Modeling (MLM)**: بعضی از توکن‌ها در جمله ماسک می‌شوند، و مدل باید آن‌ها را پیش‌بینی کند.
  * **Next Sentence Prediction (NSP)**: آیا جمله دوم دنباله جمله اول هست یا نه؟
* **کاربردها**: درک زبان، طبقه‌بندی متن، Named Entity Recognition و ...

---

### 📌 2. **RoBERTa (Robustly Optimized BERT Pretraining Approach)**

* **نوع**: Encoder-only (مبتنی بر BERT)
* **ایده اصلی**: بهبود کیفیت آموزش BERT با حذف NSP، استفاده از داده بیشتر و batchهای بزرگ‌تر.
* **تفاوت با BERT**:

  * فقط از MLM استفاده می‌کند.
  * از shuffling بهتر داده‌ها و dynamic masking بهره می‌برد.
* **کاربردها**: مشابه BERT اما با دقت بالاتر در بسیاری از تست‌ها.

---

### 📌 3. **ELECTRA**

* **نوع**: Encoder-only (ولی با مکانیزم متفاوت)
* **ایده اصلی**: به‌جای پیش‌بینی توکن‌های ماسک‌شده (مانند BERT)، یاد می‌گیرد تشخیص دهد که **آیا یک توکن واقعی است یا جایگزین شده**.
* **روش آموزش**:

  * یک generator توکن‌های جعلی تولید می‌کند.
  * یک discriminator یاد می‌گیرد تشخیص دهد کدام توکن‌ها تغییر کرده‌اند.
* **مزیت**: سریع‌تر و کارآمدتر از BERT از نظر یادگیری.

---

## 🔹 خانواده Decoder-only (مدل‌هایی مثل GPT)

### 📌 1. **GPT-2**

* **نوع**: Decoder-only
* **ایده اصلی**: مدل‌سازی زنجیره‌ای زبان به‌صورت **چپ به راست** (causal), با پیش‌بینی توکن بعدی.
* **روش آموزش**:

  * فقط از **Language Modeling (Auto-regressive)** استفاده می‌کند.
* **کاربردها**: تولید متن، خلاصه‌سازی، پاسخ به پرسش، ترجمه و ...

---

### 📌 2. **GPT-3**

* **نوع**: Decoder-only (توسعه‌یافته‌تر از GPT-2)
* **ایده اصلی**: مقیاس‌دهی شدید (175B پارامتر) + توانایی انجام چندوظیفه‌ای بدون fine-tuning.
* **ویژگی خاص**: یادگیری از طریق نمونه (Few-shot / Zero-shot Learning).
* **کاربردها**: چت‌بات، برنامه‌نویسی خودکار، مقاله‌نویسی و...

---

### 📌 3. **LLaMA (Large Language Model Meta AI)**

* **نوع**: Decoder-only
* **ایده اصلی**: طراحی مدل‌های سبک‌تر اما بسیار کارآمد (معمولاً با پارامترهای کمتر از GPT-3 اما عملکرد مشابه).
* **مزیت**: باز بودن مدل‌ها و توانایی اجرا روی سخت‌افزار محدود.
* **کاربردها**: چت‌بات، تحقیقاتی، ابزارهای پردازش زبان باز.

---

## ✅ جمع‌بندی جدول‌وار:

| مدل     | نوع معماری   | روش یادگیری اصلی           | کاربرد کلیدی                |
| ------- | ------------ | -------------------------- | --------------------------- |
| BERT    | Encoder-only | MLM + NSP                  | درک زبان                    |
| RoBERTa | Encoder-only | MLM (بهینه‌شده)            | طبقه‌بندی و درک پیشرفته     |
| ELECTRA | Encoder-only | Discriminator-Generator    | آموزش سریع‌تر و دقیق‌تر     |
| GPT-2   | Decoder-only | Auto-regressive LM         | تولید متن                   |
| GPT-3   | Decoder-only | Few-shot + Auto-regressive | پاسخ‌دهی چندکاره            |
| LLaMA   | Decoder-only | Efficient auto-regressive  | تحقیقاتی و کاربردهای سبک‌تر |

---

اگر خواستی مدل‌هایی از خانواده Encoder-Decoder مثل T5 یا BART رو هم بررسی کنیم، فقط بگو.


🦾 تمرین

با چه تغییراتی می‌توان GPT را به مدلی مشابه Llama 3.2 تبدیل کرد؟ ایده‌های خود را بیان کنید.

این کد یکی از بخش‌های مهم **معماری مدل LLaMA 3** از شرکت **Meta AI** را پیاده‌سازی می‌کند. این معماری از نوع **Decoder-only** و بر پایه Transformer است، با بهینه‌سازی‌هایی مانند **Rotary Embeddings**، **مدیریت حافظه (KV Caching)**، و **مدل‌سازی موازی (Model Parallelism)** با استفاده از کتابخانه `fairscale`.

بیایید آن را گام به گام و به زبان ساده تحلیل کنیم:

---

## 🧩 ساختار کلی کد

| بخش                                                         | توضیح                                                        |
| ----------------------------------------------------------- | ------------------------------------------------------------ |
| `RMSNorm`                                                   | نرمال‌سازی RMS، جایگزینی سبک‌تر و پایدارتر برای LayerNorm    |
| `apply_scaling`, `precompute_freqs_cis`, `apply_rotary_emb` | پیش‌پردازش موقعیت‌یابی چرخشی (RoPE)                          |
| `Attention`                                                 | پیاده‌سازی کامل Attention با کش (KV cache) و پشتیبانی از GQA |
| `FeedForward`                                               | شبکه FFN دو شاخه‌ای با سیگموید غیرخطی (SiLU)                 |
| `TransformerBlock`                                          | یک بلوک کامل شامل Attention + FFN                            |
| `Transformer`                                               | مدل کامل LLaMA شامل لایه‌های متعدد، embedding و خروجی نهایی  |

---

## 🔹 جزئیات مهم هر بخش

### 🧠 1. `RMSNorm`

```python
class RMSNorm(nn.Module):
```

نرمال‌سازی «ریشه میانگین مربعات»، بدون کم‌کردن میانگین. این نرمال‌سازی سریع‌تر و پایدارتر است نسبت به LayerNorm.

---

### 🌀 2. `Rotary Embeddings`

```python
def apply_rotary_emb(xq, xk, freqs_cis): ...
```

ایده RoPE این است که موقعیت توکن‌ها را به‌صورت توابع سینوسی پیچشی وارد فضای attention کنیم. این باعث می‌شود اطلاعات ترتیبی (position) در dot-product attention لحاظ شود.

* `precompute_freqs_cis` فرکانس‌ها را برای موقعیت‌ها پیش‌محاسبه می‌کند.
* `apply_rotary_emb` این فرکانس‌ها را به query و key اعمال می‌کند.

---

### ⚡ 3. `Attention`

```python
class Attention(nn.Module): ...
```

لایه attention مدل با پشتیبانی از موارد زیر:

* **GQA**: Grouped Query Attention (مثل LLaMA 2/3).
* **Model Parallelism**: با `ColumnParallelLinear` و `RowParallelLinear`.
* **KV Cache**: نگه‌داشتن key/value برای جلوگیری از محاسبه دوباره در طول decoding.
* **Rotary Embeddings**: با `apply_rotary_emb` روی xq/xk.
* **Causal Masking**: با `mask` در dot-product attention برای جلوگیری از نگاه به آینده.

---

### ⚙️ 4. `FeedForward`

```python
class FeedForward(nn.Module): ...
```

یک لایه FFN که ورودی را به hidden dimension گسترش می‌دهد و با سیگموید SiLU و ضرب pointwise خروجی تولید می‌کند. از ۳ لایه خطی استفاده شده (w1، w2، w3) برای افزایش انعطاف‌پذیری.

---

### 🧱 5. `TransformerBlock`

```python
class TransformerBlock(nn.Module): ...
```

یک بلاک کامل شامل:

* `RMSNorm → Attention → Residual`
* `RMSNorm → FFN → Residual`

با ترتیب مشابه معماری LLaMA (norm-first).

---

### 🏗️ 6. `Transformer`

```python
class Transformer(nn.Module): ...
```

مدل نهایی شامل:

* Embedding اولیه با `VocabParallelEmbedding`
* پشته‌ای از `TransformerBlock`s
* Normalization نهایی
* خروجی `ColumnParallelLinear` به اندازه‌ی واژگان

همچنین:

* `precompute_freqs_cis`: فرکانس‌های RoPE از قبل محاسبه می‌شود.
* `@torch.inference_mode()`: نشان می‌دهد که این `forward()` برای inference است.
* `mask`: برای causal attention استفاده می‌شود.

---



🦾 تمرین

مقالات Transformer، BERT، GPT-2 و GPT-3 را مطالعه کنید و خلاصه‌ای از آنها بنویسید. نیازی به مطالعه خط به خط نیست؛ مرور ایده‌های اصلی کافی است. ؟

---

## 🔷 1. **Transformer (Vaswani et al., 2017) – "Attention is All You Need"**

### 🧠 ایده‌ی اصلی:

> کنار گذاشتن ساختارهای قدیمی مثل RNN و LSTM و استفاده صرف از **مکانیزم Attention** برای مدل‌سازی دنباله‌ها (sequences).

### 📌 نکات کلیدی:

* **Self-Attention:** هر توکن با سایر توکن‌ها تعامل دارد تا زمینهٔ معنایی را بهتر درک کند.
* **Multi-Head Attention:** چندین attention موازی اجرا می‌شوند تا انواع مختلف روابط را بیاموزد.
* **بدون RNN یا CNN**: کاملاً موازی‌سازی‌شده، سریع‌تر برای آموزش.
* مدل شامل یک **encoder-decoder** با لایه‌های attention و feed-forward است.

### 🌍 تأثیر:

پایه‌گذار تمام مدل‌های بزرگ بعدی مثل BERT، GPT، T5 و LLaMA.

---

## 🔷 2. **BERT (Devlin et al., 2018) – "Bidirectional Encoder Representations from Transformers"**

### 🧠 ایده‌ی اصلی:

> مدل زبانی **دوطرفه (bidirectional)** که کل جمله را (قبل و بعد) درک می‌کند؛ آموزش به‌صورت از‌پیش (pretraining) و بعد استفاده در وظایف مختلف.

### 📌 نکات کلیدی:

* از **encoder**های ترنسفورمر استفاده می‌کند (نه decoder).
* **Masked Language Modeling (MLM):** در حین آموزش، برخی توکن‌ها پنهان می‌شوند و مدل باید آن‌ها را حدس بزند.
* **Next Sentence Prediction (NSP):** تشخیص دهد که آیا دو جمله‌ی پشت‌سرهم به هم مربوط‌اند یا نه.
* مناسب برای **درک متن**، نه تولید متن.

### 🌍 تأثیر:

انقلابی در **درک زبان** (مثلاً پاسخ‌ به سوال، طبقه‌بندی، ترجمه). الهام‌بخش RoBERTa، ALBERT، و DistilBERT.

---

## 🔷 3. **GPT-2 (Radford et al., 2019) – "Language Models are Unsupervised Multitask Learners"**

### 🧠 ایده‌ی اصلی:

> یک مدل **زبان خودرگرسیو (Auto-regressive)** که فقط با پیش‌بینی کلمه بعدی، می‌تواند طیف وسیعی از وظایف زبانی را انجام دهد.

### 📌 نکات کلیدی:

* از **decoder-only Transformer** استفاده می‌کند (برخلاف BERT که encoder-only است).
* فقط با **language modeling معمولی** آموزش می‌بیند (بدون ماسک یا NSP).
* روی دیتاست عظیم **WebText** آموزش دیده.
* هیچ fine-tuning خاصی انجام نمی‌شود؛ فقط از **prompting** استفاده می‌شود.

### 🌍 تأثیر:

نشان داد که یک مدل بزرگ زبانی می‌تواند بدون آموزش اختصاصی، **چند وظیفه مختلف را هم‌زمان انجام دهد** (zero-shot, one-shot, few-shot).

---

## 🔷 4. **GPT-3 (Brown et al., 2020) – "Language Models are Few-Shot Learners"**

### 🧠 ایده‌ی اصلی:

> با **افزایش شدید اندازه مدل و داده‌ها**، می‌توان یادگیری few-shot را بهبود داد بدون نیاز به fine-tuning.

### 📌 نکات کلیدی:

* 175 میلیارد پارامتر (بزرگ‌ترین در زمان خود).
* همان ساختار GPT-2 (decoder-only, auto-regressive) ولی بسیار بزرگ‌تر.
* فقط با prompting می‌توان مدل را برای حل مسائلی مثل ترجمه، خلاصه‌سازی، کدنویسی، و... هدایت کرد.
* دیگر نیاز به آموزش مجدد (fine-tuning) نیست — فقط مثال بده و مدل یاد می‌گیرد.

### 🌍 تأثیر:

نشان داد که مقیاس‌دهی عظیم می‌تواند به **یادگیری عمومی و منعطف** منجر شود؛ نقطهٔ عطف در ساخت مدل‌های مولد عمومی مثل ChatGPT و Copilot.

---

## 📊 مقایسه کوتاه:

| مدل         | ساختار            | نوع یادگیری                 | هدف               | کاربرد اصلی               |
| ----------- | ----------------- | --------------------------- | ----------------- | ------------------------- |
| Transformer | Encoder + Decoder | Supervised (ترجمه)          | معماری پایه       | پایهٔ همه مدل‌های بعدی    |
| BERT        | Encoder-only      | Pretraining + Fine-tuning   | درک زبان          | طبقه‌بندی، پاسخ‌به‌سوال   |
| GPT-2       | Decoder-only      | فقط Pretraining             | تولید متن         | متن‌نویسی، multi-task     |
| GPT-3       | Decoder-only      | فقط Pretraining + Prompting | few-shot learning | یادگیری منعطف، general AI |

---

اگر بخوای، می‌تونم این اطلاعات رو توی جدول PDF یا نمودار گرافیکی هم دربیارم، یا خلاصه انگلیسی‌شون رو هم ارائه بدم.
